<a href="https://colab.research.google.com/github/arashfahim/Stochastic_Control_FSU/blob/main/control_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modules

functions and file names such as ```my_file.py``` use lowercase letters and ```_```.

In [26]:
def step_loss(t,x,model):
    # fix concatinate t and x before plugging it into the model
    T = torch.ones_like(x)*t*dt
    tx = torch.concat((T,x),dim=1)
    u = model(tx)
    return torch.mean(x**2+x+u**2)*dt

def total_loss(model):
    x = x0
    running_cost = 0.0
    for t in range(N):
        # fix concatinate t and x before plugging it into the model
        T = torch.ones_like(x)*t*dt
        tx = torch.concat((T,x),dim=1)
        u = model(tx)
        running_cost = running_cost + step_loss(t,x,model)
        x = update(t,x,u)
    terminal_cost = torch.mean(x**2 - x)
    total_loss = running_cost + terminal_cost
    return total_loss

# GLOBAL VARIABLES

Global variables use all caps.

```GOLDEN_RATIO = (1+np.sqrt(5))/2 ```

In [27]:
import numpy as np
GOLDEN_RATION = (1+np.sqrt(5))/2

# Classes

CapWords

In [28]:
class MyClass:
    def __init__(self, value):
        self.value = value

m = MyClass(10)

# Exceptions

In [ ]:
class CustomError(Exception):
    def __init__(self, message):
        super().__init__(message)

# Creating an instance of CustomError
custom_exception = CustomError("This is a custom error message")

# Catching and handling the exception
try:
    x = int(input("0"))
    # break
    raise custom_exception
except CustomError as ce:
    print(f"Caught a custom exception: {ce}")

# Other considerations

- Don't write loose codes. Always wrap your code in a function.

- Write codes top down.Write the most important pieces first by assuming smaller parts are established.

{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "3ebd46f4",
   "metadata": {},
   "source": [
    "# Solving optimal control problems with policy gradient method "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 104,
   "id": "1a2b3c",
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import time\n",
    "import matplotlib\n",
    "import matplotlib.pyplot as plt\n",
    "import matplotlib.font_manager as font_manager\n",
    "import matplotlib.patches as patches\n",
    "font = font_manager.FontProperties(style='normal', size=20)\n",
    "plt.rc('text', usetex=True)\n",
    "plt.rc('text.latex', preamble=r'\\usepackage{amsmath}')\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.nn.functional as F\n",
    "import torch.optim as optim\n",
    "torch.set_default_dtype(torch.float32) # improved the speed when the parameters are float32\n",
    "import random\n",
    "\n",
    "import json\n",
    "\n",
    "from IPython.display import display, Math, Markdown\n",
    "\n",
    "import datetime\n",
    "import os\n",
    "timestamp = datetime.datetime.now().strftime(\"%Y-%m-%d_%H-%M\")\n",
    "timestamp\n",
    "version = '_0.1.0'\n",
    "import math\n",
    "PI = math.pi"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "dfb9fdfd",
   "metadata": {},
   "source": [
    "## Defining control by a neural network\n",
    "\n",
    "$$\\phi(t,x;\\theta)$$\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 105,
   "id": "4dd6e905",
   "metadata": {},
   "outputs": [],
   "source": [
    "num_neurons = 20 # Modify the number of neurons\n",
    "\n",
    "model = torch.nn.Sequential(\n",
    "            torch.nn.Linear(2, num_neurons),\n",
    "            torch.nn.ReLU(),\n",
    "            torch.nn.Linear(num_neurons,1),\n",
    "        )"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "18af3ded",
   "metadata": {},
   "source": [
    "## Predefine global variables"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 106,
   "id": "d3a83fcc",
   "metadata": {},
   "outputs": [],
   "source": [
    "N = 10 # number of time steps\n",
    "T = 1.0 # terminal time\n",
    "dt = T/N # time step\n",
    "num_samples = 1_000 # Modify the number of samples\n",
    "x0 = 2*torch.rand([num_samples,1]) -1 # initial condition uniformly random in [-1,1]\n",
    "dW = torch.sqrt(torch.tensor(dt))*torch.randn([num_samples,N,1]) # Brownian increments"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ffaf6f6b",
   "metadata": {},
   "source": [
    "## State variable\n",
    "\n",
    "$$dX_t = (x_t-u_t)dt + dW_t$$\n",
    "\n",
    "$$u_t=\\phi(t,x_t,\\theta)$$"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 107,
   "id": "76ca6673",
   "metadata": {},
   "outputs": [],
   "source": [
    "# update function evolution of the state over time\n",
    "def update(t,x,u):\n",
    "    return x + (x -  u)*dt + dW[:,t,:]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "70d72170",
   "metadata": {},
   "source": [
    "## Loss function \n",
    "\n",
    "$$C(x,u)=x^2+x+u^2$$\n",
    "\n",
    "$$g(x)=x^2-x$$"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 108,
   "id": "0601568f",
   "metadata": {},
   "outputs": [],
   "source": [
    "def step_loss(t,x,model):\n",
    "    # fix concatinate t and x before plugging it into the model\n",
    "    T = torch.ones_like(x)*t*dt\n",
    "    tx = torch.concat((T,x),dim=1)\n",
    "    u = model(tx)\n",
    "    return torch.mean(x**2+x+u**2)*dt\n",
    "\n",
    "def total_loss(model):\n",
    "    x = x0\n",
    "    running_cost = 0.0\n",
    "    for t in range(N):\n",
    "        # fix concatinate t and x before plugging it into the model\n",
    "        T = torch.ones_like(x)*t*dt\n",
    "        tx = torch.concat((T,x),dim=1)\n",
    "        u = model(tx)\n",
    "        running_cost = running_cost + step_loss(t,x,model)\n",
    "        x = update(t,x,u)\n",
    "    terminal_cost = torch.mean(x**2 - x)\n",
    "    total_loss = running_cost + terminal_cost\n",
    "    return total_loss"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c780775d",
   "metadata": {},
   "source": [
    "## Generate the number of samples"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 109,
   "id": "c386b8e6",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "1 tensor(5.6927, grad_fn=<AddBackward0>)\n",
      "11 tensor(2.6294, grad_fn=<AddBackward0>)\n",
      "21 tensor(2.6715, grad_fn=<AddBackward0>)\n",
      "31 tensor(2.4921, grad_fn=<AddBackward0>)\n",
      "41 tensor(2.4447, grad_fn=<AddBackward0>)\n",
      "51 tensor(2.4461, grad_fn=<AddBackward0>)\n",
      "61 tensor(2.4288, grad_fn=<AddBackward0>)\n",
      "71 tensor(2.4242, grad_fn=<AddBackward0>)\n",
      "81 tensor(2.4196, grad_fn=<AddBackward0>)\n",
      "91 tensor(2.4155, grad_fn=<AddBackward0>)\n"
     ]
    }
   ],
   "source": [
    "num_epochs = 100\n",
    "learning_rate = 1e-2\n",
    "# optimizer = optim.Adam(model.parameters(), learning_rate)\n",
    "optimizer = optim.SGD(model.parameters(), learning_rate,momentum=0.9)\n",
    "loss_history = []\n",
    "for e in range(num_epochs):\n",
    "    loss = total_loss(model)#.clone().detach().requires_grad_(True)\n",
    "    optimizer.zero_grad() # Zero the gradients before running the backward pass.\n",
    "    loss.backward() # Backward pass: compute gradient of the loss with respect to all the learnable parameters\n",
    "    loss_history.append(loss.item())\n",
    "    if e % (num_epochs//10) == 0:\n",
    "        print(e+1, loss)\n",
    "    optimizer.step() # Update the weights and biases using gradient descent. Each parameter is a Tensor. Equivalent to the above three lines, but more efficient."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 110,
   "id": "d3364299",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(loss_history)\n",
    "plt.yscale('log')\n",
    "plt.xlabel('Epochs',fontproperties=font)\n",
    "plt.ylabel('Loss',fontproperties=font)\n",
    "plt.title('Loss history',fontproperties=font)\n",
    "plt.grid()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1646f730",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "788d81b8",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "base",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
